<a href="https://colab.research.google.com/github/YeralyK/TTS_Learning/blob/main/RussianTTS(SpeechT5_FineTuning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers datasets soundfile accelerate speechbrain==0.5.16

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import load_dataset, Audio
#https://huggingface.co/datasets/0x7o/klara-voice
dataset = load_dataset("0x7o/klara-voice", split="train")
dataset

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

data/train-00000-of-00018.parquet:   0%|          | 0.00/482M [00:00<?, ?B/s]

data/train-00001-of-00018.parquet:   0%|          | 0.00/535M [00:00<?, ?B/s]

data/train-00002-of-00018.parquet:   0%|          | 0.00/500M [00:00<?, ?B/s]

...

data/train-00016-of-00018.parquet:   0%|          | 0.00/517M [00:00<?, ?B/s]

data/train-00017-of-00018.parquet:   0%|          | 0.00/607M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29142 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'text'],
    num_rows: 29142
})

In [8]:
len(dataset)

29142

In [9]:
print(dataset)

Dataset({
    features: ['audio', 'text'],
    num_rows: 29142
})


In [10]:
half_size = len(dataset) // 3

dataset = dataset.select(range(half_size))

print(dataset)

Dataset({
    features: ['audio', 'text'],
    num_rows: 9714
})


In [11]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [12]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [13]:
tokenizer = processor.tokenizer

In [14]:
dataset[2:5]

{'audio': [<datasets.features._torchcodec.AudioDecoder at 0x7a9f184aa060>,
 'text': ['Есть ли в этом логика? Конечно нет. Поэтому вынимаете голову из песка или попросите кого-то помочь вам это сделать и встретить с правдой лицом к лицу, какой бы некомфортной она ни была.',
  'На все руки мастер. Вы пытаетесь усидеть на двух стульях сразу, к примеру работаете над несколькими бизнес-идеями.',
  'У Момта вы понимаете, что лучше сосредоточиться на одной, но вам жалко бросать другие начинания. Вы боитесь потерять прогресс и затраченное время.']}

In [15]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"audio": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["audio"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/9714 [00:00<?, ? examples/s]

In [16]:
dataset_vocab - tokenizer_vocab

{' ',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'Й',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Э',
 'Ю',
 'Я',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё'}

In [17]:
import re

def normalize_text(text):
    text = text.lower()

    text = re.sub(r'[^\w\s\']', '', text)

    text = ' '.join(text.split())

    return text

def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['text'])
    return example

dataset = dataset.map(add_normalized_text)

print(dataset[2:5])

Map:   0%|          | 0/9714 [00:00<?, ? examples/s]

{'audio': [<datasets.features._torchcodec.AudioDecoder object at 0x7aa2e2d4ddc0>, <datasets.features._torchcodec.AudioDecoder object at 0x7aa2e9ca4440>, <datasets.features._torchcodec.AudioDecoder object at 0x7aa2e2d23620>], 'text': ['Есть ли в этом логика? Конечно нет. Поэтому вынимаете голову из песка или попросите кого-то помочь вам это сделать и встретить с правдой лицом к лицу, какой бы некомфортной она ни была.', 'На все руки мастер. Вы пытаетесь усидеть на двух стульях сразу, к примеру работаете над несколькими бизнес-идеями.', 'У Момта вы понимаете, что лучше сосредоточиться на одной, но вам жалко бросать другие начинания. Вы боитесь потерять прогресс и затраченное время.'], 'normalized_text': ['есть ли в этом логика конечно нет поэтому вынимаете голову из песка или попросите когото помочь вам это сделать и встретить с правдой лицом к лицу какой бы некомфортной она ни была', 'на все руки мастер вы пытаетесь усидеть на двух стульях сразу к примеру работаете над несколькими бизне

In [18]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"audio": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["audio"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/9714 [00:00<?, ? examples/s]

In [19]:
dataset_vocab - tokenizer_vocab

{' ',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё'}

In [20]:
replacements = [
    ("а", "a"),
    ("б", "b"),
    ("в", "v"),
    ("г", "g"),
    ("д", "d"),
    ("е", "e"),
    ("ё", "yo"),
    ("ж", "zh"),
    ("з", "z"),
    ("и", "i"),
    ("й", "y"),
    ("к", "k"),
    ("л", "l"),
    ("м", "m"),
    ("н", "n"),
    ("о", "o"),
    ("п", "p"),
    ("р", "r"),
    ("с", "s"),
    ("т", "t"),
    ("у", "u"),
    ("ф", "f"),
    ("х", "kh"),
    ("ц", "ts"),
    ("ч", "ch"),
    ("ш", "sh"),
    ("щ", "shch"),
    ("ъ", ""),      # hard sign removed
    ("ы", "y"),
    ("ь", ""),      # soft sign removed
    ("э", "e"),
    ("ю", "yu"),
    ("я", "ya"),
]
def cleanup_text(inputs):
    text = inputs["normalized_text"]
    for src, dst in replacements:
        text = text.replace(src, dst)
    inputs["normalized_text"] = text
    return inputs

dataset = dataset.map(cleanup_text)

Map:   0%|          | 0/9714 [00:00<?, ? examples/s]

In [21]:
import os
import torch

import huggingface_hub

if not hasattr(huggingface_hub, "_orig_hf_hub_download"):
    huggingface_hub._orig_hf_hub_download = huggingface_hub.hf_hub_download

def _hf_hub_download_compat(repo_id, filename, *args, **kwargs):
    if "use_auth_token" in kwargs and "token" not in kwargs:
        kwargs["token"] = kwargs.pop("use_auth_token")
    else:
        kwargs.pop("use_auth_token", None)

    if repo_id == "speechbrain/spkrec-xvect-voxceleb" and filename == "custom.py":
        stub_dir = os.path.join("/tmp", "speechbrain_hf_stubs", "speechbrain_spkrec_xvect")
        os.makedirs(stub_dir, exist_ok=True)
        stub_path = os.path.join(stub_dir, "custom.py")
        if not os.path.exists(stub_path):
            with open(stub_path, "w", encoding="utf-8") as f:
                f.write("# Stub file auto-created to satisfy SpeechBrain downloader.\n")
        return stub_path

    return huggingface_hub._orig_hf_hub_download(repo_id, filename, *args, **kwargs)

huggingface_hub.hf_hub_download = _hf_hub_download_compat

from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
device = "cuda" if torch.cuda.is_available() else "cpu"

speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", "speechbrain_spkrec_xvect"),
)

def create_speaker_embedding(waveform):
    with torch.no_grad():
        wav = waveform if torch.is_tensor(waveform) else torch.tensor(waveform)
        wav = wav.float()

        # expected shape: [batch, time]
        if wav.ndim == 1:
            wav = wav.unsqueeze(0)
        elif wav.ndim != 2:
            raise ValueError(f"Expected 1D or 2D waveform, got shape {tuple(wav.shape)}")

        wav = wav.to(device)

        emb = speaker_model.encode_batch(wav)                 # usually [B, 1, D]
        emb = torch.nn.functional.normalize(emb, dim=2)
        return emb.squeeze().cpu().numpy()


hyperparams.yaml: 0.00B [00:00, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt: 0.00B [00:00, ?B/s]

In [22]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    example["labels"] = example["labels"][0]

    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [23]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [24]:
processed_example["speaker_embeddings"].shape

(512,)

In [25]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/9714 [00:00<?, ? examples/s]

In [26]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

Filter:   0%|          | 0/9714 [00:00<?, ? examples/s]

9709

In [27]:
dataset = dataset.train_test_split(test_size=0.1)

In [28]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        del batch["decoder_attention_mask"]

        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [29]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [30]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/396 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/585M [00:00<?, ?B/s]

SpeechT5ForTextToSpeech LOAD REPORT from: microsoft/speecht5_tts
Key                                         | Status     |  | 
--------------------------------------------+------------+--+-
speecht5.decoder.prenet.encode_positions.pe | UNEXPECTED |  | 
speecht5.encoder.prenet.encode_positions.pe | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [31]:
from functools import partial

model.config.use_cache = False

model.generate = partial(model.generate, use_cache=True)

In [32]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_russian_speech",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

In [33]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    processing_class=processor,
)


In [34]:
trainer.train()

Step,Training Loss,Validation Loss
100,5.097177,0.585762
200,4.650565,0.518918
300,4.414843,0.497781
400,4.259202,0.486158
500,4.196097,0.480874


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=500, training_loss=4.733791458129883, metrics={'train_runtime': 1146.073, 'train_samples_per_second': 13.961, 'train_steps_per_second': 0.436, 'total_flos': 1189233638445600.0, 'train_loss': 4.733791458129883, 'epoch': 1.8274599542334096})

In [35]:
trainer.push_to_hub()

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ..._speech/training_args.bin: 100%|##########| 5.39kB / 5.39kB            

  ...ian_speech/spm_char.model: 100%|##########|  238kB /  238kB            

  ...911038.418edc2f95ef.244.0: 100%|##########| 11.8kB / 11.8kB            

  ..._speech/model.safetensors:   6%|5         | 33.5MB /  578MB            

CommitInfo(commit_url='https://huggingface.co/YeralyKaimolda/speecht5_finetuned_russian_speech/commit/7008a37ee303836ff6a6f3bafab07e013c56148f', commit_message='End of training', commit_description='', oid='7008a37ee303836ff6a6f3bafab07e013c56148f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/YeralyKaimolda/speecht5_finetuned_russian_speech', endpoint='https://huggingface.co', repo_type='model', repo_id='YeralyKaimolda/speecht5_finetuned_russian_speech'), pr_revision=None, pr_num=None)

In [36]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "speecht5_finetuned_russian_speech"
)

Loading weights:   0%|          | 0/396 [00:00<?, ?it/s]

In [37]:
example = dataset["test"][304]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [38]:
text = "Привет, меня зовут Ералы, я студент университета"

In [41]:
import re

RU_DIGITS = {
    "0": "ноль", "1": "один", "2": "два", "3": "три", "4": "четыре",
    "5": "пять", "6": "шесть", "7": "семь", "8": "восемь", "9": "девять",
}

def replace_numbers_with_words(text: str) -> str:
    # replaces single digits only (simple but safe)
    return re.sub(r"\d", lambda m: RU_DIGITS[m.group(0)], text)


In [42]:
def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [43]:
converted_text = replace_numbers_with_words(text)
cleaned_text = cleanup_text(converted_text)
final_text = normalize_text(cleaned_text)
final_text

'пrivet menya zovut еraly ya student universiteta'

In [44]:
inputs = processor(text=final_text, return_tensors="pt")

In [45]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/158 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

In [46]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)
sf.write('output.wav', speech.numpy(), 16000)